# Interactive Analysis with Widgets

One of the most useful features of the notebook when doing data analysis are "widgets", which allow you to create on-the-fly interfaces for interactively exploring your data and changing parameter values. You can use widgets to do a lot of things, but I find the most useful thing to do with them, by far, is to interactively update model parameters and visualize the results.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
import pandas as pd

from ipywidgets import interact

---

## Basic Usage

The basic idea behind using interactive widgets is to create a function that takes in the arguments you which to manipulate interactively, and which performs some computation. To take a trivial example, we can create an interactive widget that allows us to see what the value of $x^2$ is, as a function of $x$. The function might look like this:

In [ ]:
def square(x):
    return x ** 2

Now, to turn this into an *interactive* function, we need to do two things.

1. First, we need to specify the range of what $x$ can be. For our purposes, let's say $x$ can be between -10 and 10, and that we'll consider intervals of 0.1.
2. Second, we need to actually create a widget out of this function.

We can do both of these things using the `interact` function:

In [ ]:
interact(square, x=(-10, 10, 0.1));

Once you execute the above cell, you'll see a slider labeled with "x" and a number printed out beneath. Try changing the value of the slider and you'll see the number change!

This is the basic idea behind widgets: create a function, and then pass it to the `interact` function along with the range of values for the function's arguments.

We can also create widgets using the "decorator" version of `interact`, e.g.:

In [ ]:
@interact(x=(-10, 10, 0.1))
def square(x):
    return x ** 2

---

## Exploring the PDF of a Random Variable

We can use widgets to create much more interesting visualizations and interactive behavior. To take a more complex, yet still simple example, we can plot the PDF of a Gaussian (normal) distribution and explore what happens when we modify the parameters for mean and standard deviation. Here, we'll use the same ideas as we saw when we created animated plots using matplotlib.

In [ ]:
# set up the plot
plt.close('all')
fig, ax = plt.subplots()
x = np.linspace(-4, 4, 1000)
line, = ax.plot([], [])
ax.set_xlim(-4, 4)
ax.set_ylim(0, 2)
    
@interact(mu=(-2, 2, 0.01), sigma=(0.01, 2, 0.01))
def plot_normal_distribution(mu, sigma):
    y = scipy.stats.norm.pdf(x, mu, sigma)
    line.set_xdata(x)
    line.set_ydata(y)
    ax.set_title("mu={}, sigma={}".format(mu, sigma))
    fig.canvas.draw()

This receipe can be really useful, especially for exploring other types of distributions that you might not be so familiar with, such as the beta distribution, which is parameterized by $\alpha$ and $\beta$:

In [ ]:
# set up the plot
plt.close('all')
fig, ax = plt.subplots()
x = np.linspace(0, 1, 1000)
line, = ax.plot([], [])
ax.set_xlim(0, 1)
ax.set_ylim(0, 10)
    
@interact(alpha=(0, 10, 0.1), beta=(0, 10, 0.1))
def plot_beta_distribution(alpha, beta):
    y = scipy.stats.beta.pdf(x, alpha, beta)
    line.set_xdata(x)
    line.set_ydata(y)
    ax.set_title("alpha={}, beta={}".format(alpha, beta))
    fig.canvas.draw()

---

## More Complex Widgets

We can create much more complex widgets than just ones with sliders. For example, let's consider again our bouncing ball experiment dataset:

In [ ]:
data = pd.read_csv("data/ball.csv")

# filter out extreme response times
lo, hi = np.percentile(data["rt"], [0.5, 99.5])
data = data.query("rt > {} and rt < {}".format(lo, hi))

data.head()

We can get a better feel for how responses and response times vary as a function of the stimulus by creating an interactive widget to quickly flip through the different stimuli.

In [ ]:
plt.close('all')
fig, (ax1, ax2) = plt.subplots(1, 2)

@interact(stim=data["stim"].unique(), hole_width=data["hole_width"].unique(), hole_class=data["hole_class"].unique())
def plot(stim, hole_width, hole_class):
    df = data.query("stim == '{}' and hole_width == {} and hole_class == '{}'".format(stim, hole_width, hole_class))

    # plot a histogram of responses
    ax1.clear()
    bins = [1 - df["response"].mean(), df["response"].mean()]
    ax1.bar([0, 1], bins, edgecolor='w')
    ax1.set_ylim(0, 1)
    ax1.set_xticks([0, 1])
    ax1.set_xticklabels([False, True])
    ax1.set_title("Responses")

    # plot a histogram of log response times
    ax2.clear()
    ax2.hist(np.log(df["rt"]), bins=20, range=[-3, 3], normed=True)
    ax2.set_ylim(0, 1)
    ax2.set_title("Log Response Times")
    
    # fit a normal distribution to the log response times and plot it
    x = np.linspace(-3, 3)
    mu = np.log(df["rt"]).mean()
    std = np.log(df["rt"]).std()
    y = scipy.stats.norm.pdf(x, mu, std)
    ax2.plot(x, y, 'k')

If you do computational modeling work, this is also a great way to explore where your model is closely fitting your empirical data and where it is not: plot the two measures next to each other (or on top of each other) and then move through your stimuli interactively with widgets!

---

## Exercise

<div class="alert alert-success">
Using the above widget as a template, create a widget that allows you to visualize the distribution of responses and response times for individual participants (rather than individual stimuli).
</div>

In [ ]:
# Your code here